## 包引入与变量准备

In [1]:
import sys
import csv
sys.path.append("./global")
from helper import Timer, usetime, OFFLINE, cp

In [2]:
# 原始训练与测试数据位置
if OFFLINE:
    ori_train_file = "./legacy/data/train.csv"
    ori_test_file = "./legacy/data/test.csv"
else:
    ori_train_file = "./legacy/data/train.csv"
    ori_test_file = "./legacy/data/test.csv"
    # ori_train_file = "/home/kesci/input/bytedance/first-round/train.csv"
    # ori_test_file = "/home/kesci/input/bytedance/first-round/test.csv"

In [3]:
# Stage0阶段
stage0_output_dir = "./stage1/input/"
## 训练数据相关
if OFFLINE:
    train_file = stage0_output_dir + "train.csv"
else:
    random_sample = False
    if random_sample:
        random_rate = 0.05
        train_file = stage0_output_dir + "train_random_005.csv"
    else:
        random_num = 5000000
        # 默认取前500w条
        train_file = stage0_output_dir + "train_front_500w.csv"
## 测试数据相关
test_file = stage0_output_dir + "test.csv"
if OFFLINE:
    test_label_file = stage0_output_dir + "test_label.csv"

In [4]:
# Stage1阶段
state1_dir = "./stage1/"
stage1_code_dir = state1_dir + "code/"
stage1_input_dir = state1_dir + "input/"
stage1_output_dir = state1_dir + "output/"

In [5]:
# Stage2阶段
state2_dir = "./stage2/"
stage2_input_dir = state2_dir + "input/"
stage2_output_dir = state2_dir + "output/"

## Stage0阶段

这一阶段主要是数据准备，且只需要运行一次。

In [6]:
from stage0.code.data_prepare import RandomSampleCSV, SampleCSV

In [7]:
# # 训练数据准备
# if OFFLINE:
#     cp(ori_train_file, train_file)
# else:
#     if random_sample:
#         # 随机从一亿数据中采样rate比例的数据来训练
#         RandomSampleCSV(source_csv=ori_train_file, save_file=train_file, rate=random_rate)
#     else:
#         # 取前c条数据来训练   
#         SampleCSV(source_csv=ori_train_file, save_file=train_file, count=random_num)        

In [8]:
# # 测试数据准备
# if OFFLINE:
#     cp(ori_test_file, test_file)
#     cp("./legacy/data/test_label.csv", "./stage1/input/test_label.csv")
# else:
#     cp(ori_test_file, test_file)

## Stage1阶段

### 提取词向量

#### Word2vec

1. 生成Word2vec可直接训练的sentences文件
2. 训练word2vec

In [9]:
# ProcessForTrainWord2vec(train_file_data, stage1_input_dir+"word2vec_sentences.txt")

In [10]:
# !cd stage1/code; python build_word2vec.py -f "../input/word2vec_sentences.txt" -d "../output/" -m "word2vec.model"

#### FastText

FastText的使用：https://blog.csdn.net/ymaini/article/details/81489599

1. 生成适合fastText训练的文本格式
2. 有监督训练

In [11]:
# ProcessForTrainFastText(train_file_data, stage1_input_dir+"fastText_labeled_content.txt", add_label=True)

In [12]:
# !cd stage1/code; python build_fastText.py -f "../input/fastText_labeled_content.txt" -d "../output/" -s "fastText_supervised.bin"

### 特征抽取

In [13]:
from stage1.code.feature_text import ExtractTextFeature
from helper import ORI_TRAIN_NAMES, ORI_TEST_NAMES, ORI_TRAIN_DTYPE, ORI_TEST_DTYPE
feature_prefix = "v3_nid"

#### Text Mining Featues

In [25]:
# 提取train的特征
# ExtractTextFeature(train_file, save_dir=stage1_output_dir, prefix="train_{}".format(feature_prefix), 
#                    names=ORI_TRAIN_NAMES, dtype=ORI_TRAIN_DTYPE, process_chunkly=False,
#                    drop_cols=['query', 'title', 'label', "query_id", "query_title_id"])

----->Started 'Extract text feature' block...
----->Finished 'Extract text feature' block, time used: 9.39s.


In [26]:
# 提取test的特征
# ExtractTextFeature(test_file, save_dir=stage1_output_dir, prefix="test_{}".format(feature_prefix), 
#                    names=ORI_TEST_NAMES, dtype=ORI_TEST_DTYPE, process_chunkly=False,
#                    drop_cols=['query', 'title', "query_id", "query_title_id"])

----->Started 'Extract text feature' block...
----->Finished 'Extract text feature' block, time used: 3.22s.


#### Embedding Features

In [16]:
# ! cd stage1/code; python feature_embed.py -f "../input/train_01_0607.csv" -p "train_v1" -d "../output" -b "true" -e "word2vec"
# ! cd stage1/code; python feature_embed.py -f "../input/test_copy.csv" -p "test_v1" -d "../output" -b "true" -e "word2vec" -t "false"

### Prepare for Stage2

本节是将一些stage1产生的结果进过处理后放到stage2的input里面去。

In [27]:
from stage1.code.prepare_for_stage2 import ExtractTrainLabel, ConvertCSVToNPY
# stage1
## text feature
train_text_feature_stage1 = stage1_output_dir + "train_{}_feature_text.csv".format(feature_prefix)
test_text_feature_stage1 = stage1_output_dir + "test_{}_feature_text.csv".format(feature_prefix)
# stage2
## text feature
train_text_feature_stage2 = stage2_input_dir + "train_{}_feature_text.npy".format(feature_prefix)
test_text_feature_stage2 = stage2_input_dir + "test_{}_feature_text.npy".format(feature_prefix)
train_labels_stage2 = stage2_input_dir+"train_labels.npy"
test_labels_stage2 = None

In [28]:
# 提取训练集的标签并单独存储
ExtractTrainLabel(train_file, train_labels_stage2)
if OFFLINE:
    test_labels_stage2 = stage2_input_dir+"test_labels.npy"
    ExtractTrainLabel(test_label_file, test_labels_stage2)

----->Started 'Extract label' block...
Part of labels: [1, 0, 1, 0, 1, 0, 0, 1, 0, 0]
Label file saved to ./stage2/input/train_labels.npy
----->Finished 'Extract label' block, time used: 0.04s.
----->Started 'Extract label' block...
Part of labels: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Label file saved to ./stage2/input/test_labels.npy
----->Finished 'Extract label' block, time used: 0.01s.


In [19]:
# 将特征转化为numpy格式便于读取
# ConvertCSVToNPY(train_text_feature_stage1, train_text_feature_stage2)
# ConvertCSVToNPY(test_text_feature_stage1, test_text_feature_stage2)

## Stage2阶段

In [20]:
from stage2.code.run_hyperopt import LoadDataset, run_lgb_gbdt, run_lgb_dart

/Users/niudong/workon_home/py3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [21]:
# 加载数据集
LoadDataset(in_train_feature=train_text_feature_stage2, 
            in_train_label=train_labels_stage2, 
            in_test_feature=test_text_feature_stage2,
            in_test_label=test_labels_stage2)

-------------------------------------
[Runing function]: [_load_train] 
[Used Time]: 0:00:00.010746
-------------------------------------
-------------------------------------
[Runing function]: [_load_test] 
[Used Time]: 0:00:00.009209
-------------------------------------
X: [[3.66356165 2.94443898 0.12244898 0.06521739 0.29920635 0.44801587
  3.63758616 2.89037176 0.03703704 0.01886792 0.29472349 0.38540571
  3.61091791 2.83321334 0.         0.         0.29030599 0.34875761]
 [3.66356165 2.77258872 0.25       0.14285714 0.32680443 0.43620845
  3.63758616 2.7080502  0.07843137 0.04081633 0.32240058 0.3875203
  3.61091791 2.63905733 0.         0.         0.30972785 0.36288176]] (15000, 18)
y: [1 0] (15000,)
Test x: [[1.60943791 2.07944154 0.54545455 0.375      0.54545455 0.60606061
  1.38629436 1.94591015 0.44444444 0.28571429 0.44444444 0.54901961
  1.09861229 1.79175947 0.28571429 0.16666667 0.39162562 0.41758242]
 [1.38629436 2.94443898 0.3        0.17647059 0.28571429 0.28571429
 

In [24]:
run_lgb_gbdt(eval_num=50)

{                                                   
 "bagging_fraction": 1.0,
 "bagging_freq": 18.0,
 "feature_fraction": 0.9,
 "learning_rate": 0.05,
 "max_bin": 255,
 "min_data_in_leaf": 80.0,
 "num_iterations": 120.0,
 "num_leaves": 55.0,
 "num_threads": 4
}
----->Started 'lgb fit' block...                    
[50]	valid's auc: 0.531408	valid's binary_logloss: 0.567053
[100]	valid's auc: 0.520062	valid's binary_logloss: 0.574628
----->Finished 'lgb fit' block, time used:          
0.43s.                                              
----->Started 'lgb fit' block...                    
[50]	valid's auc: 0.55065	valid's binary_logloss: 0.564306
[100]	valid's auc: 0.537906	valid's binary_logloss: 0.570553
----->Finished 'lgb fit' block, time used:          
0.36s.                                              
----->Started 'lgb fit' block...                    
[50]	valid's auc: 0.535388	valid's binary_logloss: 0.567461
[100]	valid's auc: 0.530728	valid's binary_logloss: 0.573839
----

----->Finished 'lgb fit' block, time used:                                   
0.11s.                                                                       
Valid AUC (chunk, total):0.5334669060894206 0.5375819021110599               
----->Started 'lgb fit' block...                                             
----->Finished 'lgb fit' block, time used:                                   
0.09s.                                                                       
Test AUC:0.5508056722650837                                                  
{                                                                            
 "bagging_fraction": 0.8,
 "bagging_freq": 2.0,
 "feature_fraction": 0.8,
 "learning_rate": 0.05,
 "max_bin": 63,
 "min_data_in_leaf": 70.0,
 "num_iterations": 120.0,
 "num_leaves": 35.0,
 "num_threads": 4
}
----->Started 'lgb fit' block...                                             
[50]	valid's auc: 0.547147	valid's binary_logloss: 0.563144                  
[100]	valid'

[50]	valid's auc: 0.547691	valid's binary_logloss: 0.562972                  
[100]	valid's auc: 0.54098	valid's binary_logloss: 0.565816                  
[150]	valid's auc: 0.534856	valid's binary_logloss: 0.569797                 
----->Finished 'lgb fit' block, time used:                                   
0.34s.                                                                       
----->Started 'lgb fit' block...                                             
[50]	valid's auc: 0.539533	valid's binary_logloss: 0.565106                  
[100]	valid's auc: 0.528765	valid's binary_logloss: 0.568538                 
[150]	valid's auc: 0.525851	valid's binary_logloss: 0.571772                 
----->Finished 'lgb fit' block, time used:                                   
0.32s.                                                                       
----->Started 'lgb fit' block...                                             
[50]	valid's auc: 0.539659	valid's binary_logloss: 0.563991     

Test AUC:0.528053078072982                                                   
{                                                                             
 "bagging_fraction": 0.7000000000000001,
 "bagging_freq": 18.0,
 "feature_fraction": 0.8,
 "learning_rate": 0.09,
 "max_bin": 1023,
 "min_data_in_leaf": 20.0,
 "num_iterations": 40.0,
 "num_leaves": 65.0,
 "num_threads": 4
}
----->Started 'lgb fit' block...                                              
----->Finished 'lgb fit' block, time used:                                    
0.23s.                                                                        
----->Started 'lgb fit' block...                                              
----->Finished 'lgb fit' block, time used:                                    
0.19s.                                                                        
----->Started 'lgb fit' block...                                              
----->Finished 'lgb fit' block, time used:                       

----->Started 'lgb fit' block...                                              
----->Finished 'lgb fit' block, time used:                                    
0.11s.                                                                        
----->Started 'lgb fit' block...                                              
----->Finished 'lgb fit' block, time used:                                    
0.11s.                                                                        
----->Started 'lgb fit' block...                                              
----->Finished 'lgb fit' block, time used:                                    
0.09s.                                                                        
----->Started 'lgb fit' block...                                              
----->Finished 'lgb fit' block, time used:                                    
0.09s.                                                                        
----->Started 'lgb fit' block...                    

...

0.59s.                                                                        
Test AUC:0.523273049331781                                                    
{                                                                             
 "bagging_fraction": 1.0,
 "bagging_freq": 16.0,
 "feature_fraction": 0.7000000000000001,
 "learning_rate": 0.05,
 "max_bin": 255,
 "min_data_in_leaf": 30.0,
 "num_iterations": 200.0,
 "num_leaves": 40.0,
 "num_threads": 4
}
----->Started 'lgb fit' block...                                              
[50]	valid's auc: 0.541567	valid's binary_logloss: 0.564496                   
[100]	valid's auc: 0.536385	valid's binary_logloss: 0.568815                  
[150]	valid's auc: 0.529879	valid's binary_logloss: 0.573214                  
[200]	valid's auc: 0.527738	valid's binary_logloss: 0.57669                   
----->Finished 'lgb fit' block, time used:                                    
0.38s.                                                          

{                                                                             
 "bagging_fraction": 1.0,
 "bagging_freq": 8.0,
 "feature_fraction": 0.6000000000000001,
 "learning_rate": 0.02,
 "max_bin": 255,
 "min_data_in_leaf": 60.0,
 "num_iterations": 160.0,
 "num_leaves": 40.0,
 "num_threads": 4
}
----->Started 'lgb fit' block...                                              
[50]	valid's auc: 0.544462	valid's binary_logloss: 0.562039                   
[100]	valid's auc: 0.54146	valid's binary_logloss: 0.563573                   
[150]	valid's auc: 0.537484	valid's binary_logloss: 0.56561                   
----->Finished 'lgb fit' block, time used:                                    
0.38s.                                                                        
----->Started 'lgb fit' block...                                              
[50]	valid's auc: 0.555164	valid's binary_logloss: 0.561679                   
[100]	valid's auc: 0.554155	valid's binary_logloss: 0.561777     

0.54s.                                                                        
----->Started 'lgb fit' block...                                              
[50]	valid's auc: 0.526349	valid's binary_logloss: 0.568509                   
[100]	valid's auc: 0.520392	valid's binary_logloss: 0.574697                  
[150]	valid's auc: 0.517302	valid's binary_logloss: 0.580101                  
----->Finished 'lgb fit' block, time used:                                    
0.53s.                                                                        
----->Started 'lgb fit' block...                                              
[50]	valid's auc: 0.556047	valid's binary_logloss: 0.56232                    
[100]	valid's auc: 0.547884	valid's binary_logloss: 0.566521                  
[150]	valid's auc: 0.537272	valid's binary_logloss: 0.572464                  
----->Finished 'lgb fit' block, time used:                                    
0.6s.                                               

0.27s.                                                                        
Valid AUC (chunk, total):0.5389264335702522 0.5339562943098208                
----->Started 'lgb fit' block...                                              
----->Finished 'lgb fit' block, time used:                                    
0.24s.                                                                        
Test AUC:0.539729860120588                                                    
{                                                                             
 "bagging_fraction": 1.0,
 "bagging_freq": 4.0,
 "feature_fraction": 0.8,
 "learning_rate": 0.04,
 "max_bin": 255,
 "min_data_in_leaf": 60.0,
 "num_iterations": 160.0,
 "num_leaves": 20.0,
 "num_threads": 4
}
----->Started 'lgb fit' block...                                              
[50]	valid's auc: 0.552266	valid's binary_logloss: 0.560955                   
[100]	valid's auc: 0.54422	valid's binary_logloss: 0.563644                   
[

[150]	valid's auc: 0.550661	valid's binary_logloss: 0.563919                  
[200]	valid's auc: 0.546818	valid's binary_logloss: 0.56627                   
----->Finished 'lgb fit' block, time used:                                    
0.32s.                                                                        
Valid AUC (chunk, total):0.541373656138176 0.5384098877722965                 
----->Started 'lgb fit' block...                                              
----->Finished 'lgb fit' block, time used:                                    
0.26s.                                                                        
Test AUC:0.5407775156527338                                                   
100%|██████████| 50/50 [01:45<00:00,  2.01s/it, best loss: 0.5626620124866412]


In [23]:
# run_lgb_dart(eval_num=10)